## Serve ML Model with MLflow in Docker

### 1. Train and Save the Model (MLflow format)

#### create a script : train_model.py

In [ ]:
# train_model.py
import mlflow
import mlflow.sklearn
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

# Load data
iris = load_iris()
X, y = iris.data, iris.target

# Train model
model = RandomForestClassifier()
model.fit(X, y)

# Save model in MLflow format
mlflow.sklearn.save_model(model, "model")


run it using:

In [ ]:
python train_model.py


It will create a directory model/ in MLflow format.

### 2. Create a Dockerfile to Serve the Model

In [ ]:
# Dockerfile
FROM python:3.10-slim

# Install system dependencies
RUN apt-get update && apt-get install -y curl && rm -rf /var/lib/apt/lists/*

# Install MLflow and dependencies
RUN pip install mlflow scikit-learn pandas

# Set working directory
WORKDIR /app

# Copy saved ML model
COPY model /app/model

# Expose MLflow's default port
EXPOSE 5000

# Start MLflow model server
CMD ["mlflow", "models", "serve", "-m", "model", "-h", "0.0.0.0", "-p", "5000"]


Build Docker Image

In [ ]:
docker build -t mlflow-model-server .


Run the Container

In [ ]:
docker run -p 5000:5000 mlflow-model-server


This starts the MLflow server inside the container, serving the model at:

In [ ]:
http://localhost:5000/invocations

### Test Model Inference (via curl or Postman)

In [ ]:
curl -X POST http://localhost:5000/invocations \
-H "Content-Type: application/json" \
-d '{"columns":["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"],"data":[[5.1, 3.5, 1.4, 0.2]]}'


#### Folder Structure

In [ ]:
your_project/
├── Dockerfile
├── train_model.py
└── model/
    ├── MLmodel
    ├── conda.yaml
    └── artifacts.pkl (etc.)
